# Pixasonics: An Image Sonification Toolbox for Python

# Introduction

Pixasonics is a library for interactive audiovisual image analysis and exploration, through image sonification. That is, it is using real-time audio and visualization to listen to image data: to map between image features and acoustic parameters. This can be handy when you need to work with a large number of images, image stacks, or hyper-spectral images (involving many color channels) where visualization becomes limiting, challenging, and potentially overwhelming.

With pixasonics, you can launch a little web application (running in a Jupyter notebook), where you can load images, probe their data with various feature extraction methods, and map the extracted features to parameters of synths, devices that make sound. You can do all this in real-time, using a visual interface, you can remote-control the interface programmatically, record sound real-time, or non-real-time, with a custom script.

You can also run multiple instances of the app, or use it "headless mode" (without its visual interface), perhaps in a command-line script.

## If you are in a hurry...

In [ ]:
# ;pip install pixasonics

# quick workflow with a simple example
from pixasonics.core import App, Mapper
from pixasonics.features import MeanChannelValue
from pixasonics.synths import Theremin

# create a new app
app = App() # by default 500x500 pixels

# load an image from file
app.load_image_file("images/test.jpg")

# create a Feature that will report the mean value of the red channel
mean_red = MeanChannelValue(filter_channels=0, name="MeanRed")
# attach the feature to the app
app.attach(mean_red) # this adds it to the pipeline, so that it is updated every frame (the Probe moves)

# create a Theremin, a simple sine wave synth that we will use to sonify the mean pixel value
theremin = Theremin(name="MySine")
# attach the Theremin to the app
app.attach(theremin) # this adds it to the pipeline, so that its audio output is patched into the global audio graph

# create a Mapper that will map the mean red pixel value (within the Probe) to the frequency of the Theremin
red2freq = Mapper(mean_red, theremin["frequency"], exponent=2, name="Red2Freq") # cubic mapping curve for a more "linear" feel of frequency changes
# attach the Mapper to the app
app.attach(red2freq) # this adds it to the pipeline, so that it is updated every frame (the Probe moves)

# Toolbox Structure

Pixasonics is mainly designed to run in a Jupyter Notebook environment. (It does also work in command line scripts.)

At the center of pixasonics is the `App` class. This represents a template pipeline where all your image data, feature extractors, synths and mappers will live. The App also comes with a graphical user interface (UI). You can do a lot with a single `App` instance, but nothing stops you from spawning different `App`s with bespoke setups.

When you have your app, you load an image (either from a file, or from a numpy array) which will be displayed in the `App` canvas. Note that _currently_ your image data height and width dimensions (the first two) will be downsampled to the `App`'s `image_size` creation argument, which is a tuple of `(500, 500)` pixels by default. (This will be improved later, stay tuned!)

Then you can explore the image data with a Probe (represented by the yellow rectangle on the canvas) using your mouse or trackpad. The Probe is your "stethoscope" on the image, and more technically, it is the sub-matrix of the Probe that is passed to all `Feature` objects in the pipeline.

Speaking of which, you can extract visual features using the `Feature` base class, or any of its convenience abstractions (e.g. `MeanChannelValue`). All basic statistical reductions are supported, such as `"mean"`, `"median"`, `"min"`, `"max"`, `"sum"`, `"std"` (standard deviation) and `"var"` (variance), but you can also make your own custom feature extractors by inheriting from the `Feature` base class (stay tuned for a K-means clustering example in the Advanced Use Cases section!). `Feature` objects also come with a UI that shows their current values and global/running min and max. There can be any number of different `Feature`s attached to the app, and all of them will get the same Probe matrix as input.

Image features are to be mapped to synthesis parameters, that is, to the settings of sound-making gadgets. (This technique is called "Parameter Mapping Sonification" in the literature.) All `Synth`s (and audio) in pixasonics are based on the fantastic [signalflow library](https://signalflow.dev/). For now, there are 5 `Synth` classes that you can use (and many more are on the way): `Theremin`, `Oscillator`, `FilteredNoise`, and `SimpleFM`. Each `Synth` comes with a UI, where you can tweak the parameters (or see them being modulated by `Mapper`s) in real-time.

What connects the output of a `Feature` and the input parameter of a Synth is a `Mapper` object. There can be multiple `Mapper`s reading from the same `Feature` buffer and a `Synth` can have multiple `Mapper`s modulating its different parameters.

# The App

The `App` class is at the core of the pixasonics workflow. The `App` is where you load your image data, where you move the probe, the `App` connects to the real-time audio server, and it represents the pipeline of `Feature`s connected to `Synth`s with `Mapper`s.

To showcase the `App` and its functionality, let's create a basic scene, using an image from the [CELLULAR open dataset](https://zenodo.org/records/8315423) and map the mean red channel value to the frequency of a Theremin (a simple sine wave generator).

In [ ]:
from pixasonics.core import App, Mapper
from pixasonics.features import MeanChannelValue
from pixasonics.synths import Theremin

# create a new app
app = App() # by default 500x500 pixels

# load an image from file
app.load_image_file("images/cellular_dataset/merged_8bit/Timepoint_001_220518-ST_C03_s1.jpg")

# create a Feature that will report the mean value of the red channel
mean_red = MeanChannelValue(filter_channels=0, name="MeanRed")
# attach the feature to the app
app.attach(mean_red) # this adds it to the pipeline, so that it is updated every frame (the Probe moves)

# create a Theremin, a simple sine wave synth that we will use to sonify the mean pixel value
theremin = Theremin()
# attach the Theremin to the app
app.attach(theremin) # this adds it to the pipeline, so that its audio output is patched into the global audio graph

# create a Mapper that will map the mean red pixel value (within the Probe) to the frequency of the Theremin
red2freq = Mapper(mean_red, theremin["frequency"], exponent=2, name="Red2Freq") # cubic mapping curve for a more "linear" feel of frequency changes
# attach the Mapper to the app
app.attach(red2freq) # this adds it to the pipeline, so that it is updated every frame (the Probe moves)

A few things just happened. The app created (or connected to) the global audio graph and the UI popped up: a canvas with the image on the left, and various settings panes on the right. The size of the canvas is determined by the `image_size` argument at the creation of the `App` object and cannot be changed afterwards. It is a tuple corresponding to `(height, width)`, and it is `(500, 500)` by default. All images that you load into the app will be __resized__ to this height and width! (This might be changed/improved in the future.) This means, that, at least currently, the app does not respect the aspect ratio of your input image, it will stretch or shrink it to whatever `image_size` your `App` has. Luckily the image we loaded is 2048x2048 with a square aspect ratio, so the default `image_size` was fine.

Now let's look to the right where the various settings panes live. These can be opened and closed, one at a time. Here is a brief summary of what you can find in them.

## Audio Settings

Click on "Audio Settings" on the top right to open this pane. Here you can control the global audio settings of the app.

### The Audio switch

There is an audio switch at the top left, and a volume slider next to it. To have any sound produced by the app, you need to turn on audio, either on the UI, or by evaluating:

In [ ]:
app.audio = True

Still no sound from the app, huh? This is because you (probably) haven't interacted with the canvas yet. Try clicking on a few of those _Drosophila_ S2 blood cells. Then try to click and drag to hear a continuous change in frequency.

### The Master Volume Slider

Next to the audio switch there is the Master Volume slider where you can set the loudness of the app's sound output in decibels (dB). You can either control it via the UI or the `master_volume` property:

In [ ]:
app.master_volume = -24 # check the slider after running this cell

### Real-time recording to file

Below the audio switch and the master slider, there is a pair of widgets that let you record the real-time output of the app. Leave the file name at the default `"recording.wav"`, hit the Record button, click and drag around the image, then click on the Record button again to stop recording.

Let's listen to what you've done:

In [ ]:
from IPython.display import Audio, display
display(Audio("recording.wav"))

The global recording is meant to be a quick-and-easy way to record your results. If you want to be more precise, (or faster than real-time) there are methods to render your results in non-real-time, which we will discuss later.

### The Master Envelope

Below the recording section you find the Master Envelope, which controls how fast the sound fades in and out when you interact with the canvas. It is a traditional Attack-Decay-Sustain-Release (ADSR) curve that is applied to the volume of the global audio output. Here is how it works in a nutshell:
- Attack: the time for the sound to fade in (in seconds),
- Decay: the time to fade to the Sustain level (in seconds),
- Sustain: the level (amplitude, between 0 and 1) to sustain indefinitely, until we release the mouse button (or deactivate the Probe),
- Release: the time for the sound to fade out after we release the mouse button (or deactivate the Probe).

There are some number fields to set these parameters, and a little drawing that visualizes the proportions of the different time segments. On the bottom, there is also a "Duration" value that shows the total duration of the envelope (that is Attack time + Decay time + Release time).

Let's set a bit slower envelope to illustrate how it works:

In [ ]:
app.master_envelope.attack = 0.5
app.master_envelope.release = 1.5

We can also set a more percussive envelope:

In [ ]:
app.master_envelope.attack = 0.01
app.master_envelope.decay = 0.01
app.master_envelope.sustain = 0.1
app.master_envelope.release = 0.8

For now, let's reset it to a more neutral default:

In [ ]:
app.master_envelope.attack = 0.1
app.master_envelope.decay = 0.01
app.master_envelope.sustain = 1
app.master_envelope.release = 0.1

In the future Envelopes will have more functionality in pixasonics, but for now there is only a global Master Envelope.

A quick technical detail while we are discussing Audio Settings. Currently, there are no controls setting the sample rate and buffer size of the audio graph (sorry!), because there are some unresolved issues about these involving the Signalflow library. For now you can get the currently used values from the `App` like so:

In [ ]:
app.sample_rate, app.output_buffer_size

Now let's close the Audio Settings pane and move on to Display Settings.

## Display Settings

By the way, we have started to stray a bit far from the cell where our app UI lives... Luckily, we can get a synced copy down here if we evaluate:

In [ ]:
app.ui

Aaah, much better. Let's open the Display Settings. As the name suggests, this is where you can find settings related to how the image is displayed in the app canvas. These will __only affect the display__ though, the underlying image data (that may be HDR, or have many color channels or image layers) will not be affected.

### Normalization

On the top of the pane you can find two checkboxes: "Normalize display" and "Global normalization". More often than not we need to normalize images to understand (or even, simply, to see!) the image content better. Traditionally, the normalization is performed individually on all color channels (Red, Green, and Blue in "normal" images). Keeping it channel-wise can help remove imbalances between the channels, like the green tint here. Check "Normalize display" in the UI, or evaluate:

In [ ]:
app.normalize_display = True

You probably noticed that the background became much less green-tinted. Let's try also checking "Global normalization":

In [ ]:
app.normalize_display_global = True

...and now the gren tint is back. It makes sense, because when normalization is set to global, the algorithm will scale all pixels according to the minimum and maximum pixel value apparent in __any__ of the channels. Since in the original image the green tint was the result of generally much higher green pixel values, applying global normalization brought the tint back. Let's turn it off for now:

In [ ]:
app.normalize_display_global = False

### Channel Offset and Layer Offset

As you see, under the checkboxes we have two (currently disabled) sliders, labelled "Channel Offset" and "Layer Offset". We will use these when we read numpy arrays instead of single image files (stay tuned!). For now, let's move on to the Probe Settings.

## Probe Settings

Close the Display Settings pane, and open the Probe Settings. This is where you set everything related to the Probe (remember, our image "stethoscope") and how you interact with it. The Probe is represented by a rectangle drawn over the image. When the Probe is active, the rectangle is red, and when the Probe is inactive, the rectangle is yellow. (Also, when the Probe is active, the `App` will evaluate all attached mappings in the pipeline and unmute audio for all attached synths.)

### The Probe Width and Height

The width and height of the Probe can vary from 1x1 pixel up to the app's `image_size`. Try moving the sliders and observe how the shape of the yellow rectangle on the canvas changes. There is not really a "right" setting for this, it is highly dependent on the content of your image, the area of pixels you want to send to feature extraction, and generally, the kind of sonification you want to create. At an extreme, you can create horizontal or vertical scan lines for the whole image like this:

In [ ]:
# horizontal scanning (a vertical scan line)
app.probe_width = 1
app.probe_height = app.image_size[0] # remember that image size is given as (height, width)

In [ ]:
# vertical scanning (a horizontal scan line)
app.probe_width = app.image_size[1]
app.probe_height = 1

Most of the time, you probably want to set the Probe dimensions to something smaller to fit the content of the image. Let's set it to fit that nice, bright cell in the middle:

In [ ]:
app.probe_width = 25
app.probe_height = 25
app.probe_x = 233
app.probe_y = 252

Yes, you guessed it: we can programmatically move the probe around using the `probe_x` and `probe_y` properties of our `App`. Setting the dimensions and the position of the probe like this will be the basis of scripting custom paths and render them non-real-time (more on that later).

### Interaction modes

Now something fun: interaction. So far we have used the "Hold" mode, that is, sound will be activated (and all existing mappers evaluated in the pipeline) while the mouse button is held down. This mode is comfortable when you want to quickly inspect (and listen to) the various parts of your image.

But sometimes you may want to activate the Probe, and then leave it on while you experiment with, let's say, changing synthesis parameters. Or you just want to free your mouse to use it somewhere else without stopping the sound output. This is what "Toggle" mode is for. You can change it on the UI by clicking on the "Toggle" button, or programmatically like so:

In [ ]:
app.interaction_mode = "toggle"

Now you can double-click on the probe to activate it. Double-click again to deactivate it. It could be sometimes comfortable to activate the Probe, and just click on various parts in the image (here, on different cells) to get an abrupt comparison of their sounding mappings. 

Try to activate the toggle and then click on the different cells to find out which one produces the highest pitch! Since we mapped the mean red channel value to the theremin frequency, and since in this image red fluorescent protein indicates the level of autophagy going on in the cell, we can intuitively find the cell with the most active autophagy by simple clicking around and listening. (Technically, if we would extract the _maximum_ red value under the probe we would get more precise results for this task, but let's give this a pass for now.)

Finally, there is a checkbox for having the Probe __always__ follow your mouse, even when you don't hold the mouse button. Beyond this being simply comfortable for your hand if you are making sound from the image continuously, it can also be handy when you don't want sound output at the moment, just want to read the value of a Feature. Let's check "Probe follows idle mouse" on the UI, or evaluate:

In [ ]:
app.probe_follows_idle_mouse = True

Now, if you hover your mouse over the app canvas, you can see that the Probe constantly follows it, and you can see indicators of the Probe's horizontal and vertical coordinates labelled "Probe X" and "Probe Y", respectively, and controlled by the `probe_x` and `probe_y` properties (as shown above).

Let's move on to the Features pane.

# Features

The Features pane is where all your `Feature` objects (that you attached to the app!) will show up. (As of now, in pixasonics, the term "feature" always refers to a _visual_ feature, as currently audio feature extraction and mapping to auditory features is not supported.) 

Right now you should see a card labelled "MeanRed" there, our Feature that reports the mean red pixel value of the Probe's slice of the image. Since we just checked the option for the Probe to follow our idle (hovering) mouse, let's move it around a bit (without necessarily making any sound), and look at the values we get on the card.

To avoid scrolling back-and-forth too much, let's have another copy of our app UI here:

In [ ]:
app.ui

Every feature gets the slice of the image data under the Probe and reduces it into one or more values. Every time you move the probe with your mouse, all the features within the `App` are sent a new Probe matrix, and they update their feature buffers (the container where they store their most-recent values). This example feature that we named "MeanRed" measures the mean value of the Probe matrix in the red color channel.

## The header

On the card you can see that, next to the name, there is a small identifier, which is uniquely generated for every different object. It is not very important now, but it might help you later identifying various objects in your pipeline.
Underneath this header you see the "Number of Features:" box that shows you how many values does this `Feature` report. Now it's just 1, since we are only filtering for the first (red) channel. Before we discuss what's behind this in more detail, let's just move on and look at the rest of the UI.

## The values

Most of the card is occupied by 3 number fields, showing the minimum, maximum and current (or most-recent) value of the feature. When a `Feature` is attached to the `App`, it actually gets to look at the whole image _once_, so it can calculate things, like global minimum and maximum values. You probably noticed that as you move the Probe around, the Min and Max fields don't change. That is because it calculated the global minimum and maximum value in the entire image.

## Resetting to running min and max

Keeping track of the range of values will come especially handy when setting up mappings. In many cases it is helpful to get global min/max values, but sometimes we might need running min/max values of the Probe matrix only. If you hover your mouse over the Reset button in the bottom left corner of the Feature card, you will see the tooltip appear: _"Reset to the running min and max"_. You can click on that or evaluate:

In [ ]:
mean_red.reset_minmax()

Notice that at the time of resetting you got the same value as min and max. But when you move the Probe around, you see that the values start to move away from each other, as the running min and max values update. This behavior can be useful when you want to create a dynamic mapping and compare two visual objects relative to each other. It can also be convenient to compare parts of the image only visually, without sonification.

## Attach to and detach from the App

But what about the other button that says "Detach"? Click on it, or evaluate:

In [ ]:
app.detach(mean_red)

Oh no, our Feature is gone! Don't worry, the `mean_red` variable still holds our `Feature` object, it's just no longer included in the `App`'s pipeline. That is, whenever the Probe moves, this detached `Feature` won't get the updated Probe matrix anymore, and its values (its feature buffer) will remain the same. Just to prove that it's still there, here is how you can pull up its UI separately:

In [ ]:
mean_red.ui

There you go, our MeanRed feature is still functional. It is just outside the `App`'s pipeline. But we can attach it back by saying:

In [ ]:
app.attach(mean_red)

If you go to the `App` UI again, you can see that now the card labelled MeanRed re-appeared in the Features pane. And by the way, our copy of the feature UI still works, it is in sync with the one in the `App` UI. Try to move the Probe and you'll see the numbers update in both places.

This mechanism of "attaching" and "detaching" objects to and from the main app is a pattern that is the same for `Feature`s, `Synth`s, and `Mapper`s. When you want to include something in the `App`, which is to say, you want to include it in the pipeline of Probe --> Features --> Mappers --> Synths, you _attach_ it to the App. When you don't need it anymore (even just for a while), you can _detach_ it from the App. The object will still "live", but it won't get updates from the `App` anymore.

Now let's create another `Feature` to see the mean pixel values across all three color channels:

In [ ]:
mean_pix = MeanChannelValue(name="MeanPix")

Recall how we created `mean_red`? It was like this:
```python
# create a Feature that will report the mean value of the red channel
mean_red = MeanChannelValue(filter_channels=0, name="MeanRed")
```

Notice that the only difference is that this time we did not specify `filter_channels` (so it defaults to `None`). By default all `Feature` objects process the entire Probe matrix which is actually a 4-dimensional matrix in the shape of:

__(Height, Width, Channels, Layers)__

_In fact, our entire image data is kept in that shape._ So when we specified `filter_channels=0`, we selected the 0th (red) channel in the 2nd, Channel dimension. Don't worry if this is a bit over your head now, the TLDR is that `Feature` objects can actually filter the Probe matrix in any way you need.

Now let's attach our new `Feature` to our `App`:

In [ ]:
app.attach(mean_pix)

Notice that the Features pane on our app UI has two cards now: MeanRed and MeanPix. We can also query the currently attached `Feature`s like so:

In [ ]:
app.features

As you have probably noticed, MeanPix reports 3 values, which are the mean pixel values across the 3 channels of the Probe matrix. We'll get into more details about `Feature`s in a bit, but for now let's move on to `Synth`s.

# Synths

`Synth`s are the tools to make sound with in pixasonics. As mentioned at the top, they are all abstractions (technically, `Patch`es) based on the awesome [signalflow library](https://signalflow.dev/).

First let's have a look at a simple Theremin synth, outside the `App`. Here is how you make one:

In [ ]:
simple_theremin = Theremin(name="SimpleTheremin")

We can pull up its UI like so:

In [ ]:
simple_theremin.ui

As you see we got a similar card here: a header with the name of the synth and its unique ID, a main section with all its parameters and corresponding sliders, as well as Reset and Detach buttons at the bottom of the card.

## The graph

There is nothing there though that turns the synth on or off. This is normally done by our `App` as a result of our interaction with the Probe. Under the hood, the `App` creates (or connects to) the global audio graph in signalflow, which is the `AudioGraph` object. This way we can use `App.graph` to reference this global audio graph and play our synth patch:

In [ ]:
graph = app.graph
graph.play(simple_theremin)

## The Parameters and Resetting

Now you should hear a static sine tone. Try moving the sliders and listen to how the sound changes to get an understanding of the parameters. When you are done, you can hit the Reset button to reset all parameters to their defaults. These defaults are not set in stone, they are decided when we create the synth object. If we spell it out, we actually said this:
```python
simple_theremin = Theremin(frequency=440, amplitude=0.5, panning=0, name="SimpleTheremin")
```

Now let's stop playing the synth:

In [ ]:
graph.stop(simple_theremin)

## Mapping to parameters

In pixasonics we currently focus on Parameter Mapping Sonification. This is probably the most common sonification technique, where data features get mapped to input parameters of a synthesizer. While this might have some perceptual issues when mappings get complex, there is a reason why this technique is so popular: it is very simple to set up and it is easily explainable.
In the future we do plan to support other types of mappings as well though, but for now, let's focus on how we can express quantities in the image (such as the level of red channel values) as changes in the parameters of synths, such as the frequency.

Let's make a new instance of our app UI again for convenience:

In [ ]:
app.ui

## Attach and detach

Open the Synths pane. Just like the Features pane, this is where all the attached `Synth`s will have their UIs. Similarly to `Feature`s, you can detach `Synth`s by clicking on the Detach button on their card, or by calling:

In [ ]:
app.detach(theremin)

This means our Theremin is not included in the `App`'s audio graph anymore, so when we activate the probe, there is no sound (since there is no `Synth` currently attached to the `App`).

We can attach the Theremin back by calling:

In [ ]:
app.attach(theremin)

## Moving sliders

Now try to activate the Probe, listen to the sound and watch the frequency slider of the Theremin. (Remember, we are still in "Toggle" interaction mode, so you have to double-click to activate the Probe.)

The slider of the Theremin seems to move on its own as we move the Probe. (Don't worry that the slider updates are not constant, this is a deliberate optimization to avoid choking the Jupyter server with slider updates. The sliders will update once you haven't moved the Probe for about 50 milliseconds.) Have you observed any relationship between the image data and the frequency of the Theremin? If you need a hint, open the Mappers pane on the app UI.

# Mappers

In our very first setup of this scene, we created and attached this `Mapper` like so:
```python
# create a Mapper that will map the mean red pixel value (within the Probe) to the frequency of the Theremin
red2freq = Mapper(mean_red, theremin["frequency"], exponent=2, name="Red2Freq") # cubic mapping curve for a more "linear" feel of frequency changes
# attach the Mapper to the app
app.attach(red2freq) # this adds it to the pipeline, so that it is updated every frame (the Probe moves)
```

`Mapper`s take two positional arguments as the `Feature` object we want to map _from_ and the `Synth` parameter we want to map _to_. The notation `theremin["frequency"]` will tell the `Mapper` to map to the frequency parameter of the theremin `Synth` object. The actual object that is returned is a dictionary:

In [ ]:
theremin["frequency"]

## Ranges at the input and output sides

There is a lot of info in here that we don't need to worry about right now. The most important for us now are the two top ones, the 'min' and 'max'.

What a `Mapper` does, is mapping a value (or an array of values) from a given input range to a given output range. It can do this either linearly (where `exponent=1`) or exponentially (like `exponent=2` for a cubic curve).

But we haven't specified any input or output range when we created our `red2freq` mapper object. How can it still know what red values to map to what frequency value?

## Default ranges

The trick is knowing that both `Feature` objects report their global (or running) min and max values (remember?!) and that all `Synth` parameter dictionaries will contain curated (sensible) min and max values. If no input and output ranges are specified upon creation, the `Mapper` object will just look up the corresponding min and max values on both the feature-side and the synth-side.

## Specific ranges

Having automatic ranges that update live with the running min/max of a `Feature` object can be handy for experimentation, but sometimes we do want to specify something more concrete. Before we get there, it is perhaps helpful to look at the spelled-out verison of how we created our `red2freq` object:
```python
red2freq = Mapper(
    obj_in=mean_red, 
    obj_out=theremin["frequency"],
    in_low=None,
    in_high=None,
    out_low=None,
    out_high=None,
    exponent=2, 
    clamp=True,
    name="Red2Freq")
```

We can specify an input range (on the `Feature` side) with the `in_low` and `out_low` arguments, while the output range (on the `Synth` parameter side) with the `out_low` and `out_high` arguments. They all default to `None`, which will mean that the `Mapper` will look up the current min/max reported by the `Feature` and the default min and max values of the `Synth` param (that were decided when we created the `Synth`).

We can also retrieve these values like so:

In [ ]:
red2freq.in_low, red2freq.in_high, red2freq.out_low, red2freq.out_high

Note that even though our MeanRed `Feature` reports singular scalar values, the data is in a Numpy array of shape `(1, 1)`. This technical detail will be more important when we want to map a vector of features to a multichannel synth, but let's ignore that case for now.

What is important is that we can now override the ranges like so:

In [ ]:
red2freq.in_low = 150
red2freq.in_high = 200

Now let's listen to what has changed in the mapping.

You probably notice that for most cells the sound remains at its minimum, while for the really bright ones it suddenly chirps up to a really high whistle. Let's go back to using the automatic ranges by setting the `in_low` and `in_high` props to `None`:

In [ ]:
red2freq.in_low = None
red2freq.in_high = None

Take a listen!

We are back to the kind of mapping we started out with. It is striking how little change resulted in such a drastic change in the sound pattern, isn't it?

## Clamp

And by the way, in the previous setting, why did all the cells that were not so bright get the same static low hum? That is because by default we clamp (limit) the mapped values to `out_low` and `out_high`. You probably want clamping to be on for most of the time, but if you know what you are doing, consider turning it off (by writing `clamp=False`) and then any input value that is below or `in_low` or above `in_high` will get proportionally mapped to below `out_low` or `out_high`. But be careful, this can quickly explode values, especially when you use exponential scaling.

## Exponent

The last important parameter of a `Mapper` is the exponent to use for the mapping. Think about it as the mapping "curve". To get a sense of how `exponent` influences the mapping curve, consider this interactive plot:

In [ ]:
from pixasonics.ui import ExponentPlot
exponent_plot = ExponentPlot()
exponent_plot()

For linear mapping we can use (the default) `exponent=1`:

In [ ]:
exponent_plot.exponent = 1

But we might want an exponential or a logarithmic curve for our mapping. In the cells below you can find some examples, but feel free to explore the curve by moving the Exponent slider!

In [ ]:
exponent_plot.exponent = 2

In [ ]:
exponent_plot.exponent = 4

In [ ]:
exponent_plot.exponent = 0.5

In [ ]:
exponent_plot.exponent = 0.25

Since we were mapping to frequencies (measured in Hz) we actually get preceptually more "linear" feel in the mapping if the curve is exponential, that is why we set `exponent=2` in our `red2freq` object.

In [ ]:
red2freq.exponent

# Advanced Use Cases

So far we have covered the basics of pixasonics, how you can set up an `App` with a pipeline that involves various `Feature`s, `Synth`s and `Mapper`s. Additionally, there are a few "breakout" doors designed to integrate pixasonics in your existing workflow or to speed/scale up your sonification sessions:
- __Loading Numpy arrays__: This lets you load any matrix data (up to 4 dimensions) as an image to sonify. If you have any specific preprocessing, you can set it up to output Numpy matrices which you can then load into an `App`. Using Numpy arrays also lets you load image sequences or hyper-spectral images (there is no conceptual restriction of the number of color channels or image layers used). By the way, if you don't want to worry about Numpy arrays, you can also directly load HDR images from files.
- __Non-real-time rendering__: Instead of having to move the Probe in real-time, perhaps for a longer recording, you can script a "timeline" and render it non-real-time. You can also reuse a script to render the same scan pattern on many images.
- __Headless mode__: While the `App` class is meant to help with interactive audiovisual exploration, you can totally skip its entire graphical user interface, and remote-control it using its properties (maybe using OSC messages coming from a different process). You should also use headless mode if you are outside of a Jupyter Notebook environment, and using pixasonics in a script.
- __Multichannel `Synth`s__: Providing a list instead of a number for any of a `Synth`s arguments will make it multichannel, which can be used to sonify `Feature`s that have more than one number. And don't worry if the number of features do not match the number of `Synth` channels: in these cases `Mapper`s will dynamically resample the feature vector to fit the number of channels.
- __`Feature` base class and custom `Feature`s__: While there are lots of convenient abstractions for simple `Feature`s (e.g. `MeanChannelValue`, `MedianRowValue`, etc), these are all just configs for the `Feature` base class, and if you learn how it works, you can intuitively fit the `Feature` to whatever slice of the image you need to focus on, using any of the "built-in" (Numpy-based) reducing methods. But you can also create your completely custom `Feature` processors (let's say one that fits a K-means model on the image) by inheriting from the `Feature` base class and overriding two of its methods.
- __Multiple `App`s in the same session__: You can also set up different `App`s with different pipelines (or even images) and use them simultaneously in the same Notebook. For scientists, this can help testing the same sonifications on different images (or sequences), or different sonification setups on the same image data. For creatives, this will let you create different interactive instruments.

Before we jump into the advanced examples, let's import everything we'll need.

In [1]:
import numpy as np
import os
from PIL import Image
from IPython.display import Audio, display
from pixasonics.core import App, Mapper
from pixasonics.features import Feature, MeanChannelValue
from pixasonics.synths import Theremin, Oscillator

## Loading HDR images and Numpy arrays

### HDR images from files

In [ ]:
# Create app
app = App()

In [ ]:
# load HDR images
img_path = "images/cellular_dataset/single_channel_16bit/Timepoint_005_220518-ST_C03_s1_w1.TIF"
app.load_image_file(img_path)

### Numpy arrays

In [ ]:
# load image as a numpy array
img_path = "images/cellular_dataset/merged_8bit/Timepoint_005_220518-ST_C03_s1.jpg"
img = Image.open(img_path)
img = np.array(img)
app.load_image_data(img) # load as numpy array

In [ ]:
# combine two HDR images and load as numpy array
img_path = "images/cellular_dataset/single_channel_16bit/Timepoint_005_220518-ST_C03_s1_w2.TIF"
img_path2 = "images/cellular_dataset/single_channel_16bit/Timepoint_005_220518-ST_C03_s1_w1.TIF"
img = Image.open(img_path)
img2 = Image.open(img_path2)
img = np.array(img)
img2 = np.array(img2)
img = np.stack([img, img2], axis=-1)
print(img.shape)
app.load_image_data(img) # load as numpy array

In [ ]:
# read all red-ch images into arrays and concatenate them in the channel dimension
img_folder = "images/cellular_dataset/single_channel_16bit/"
img_files = sorted(os.listdir(img_folder))
img_files = [f for f in img_files if f.endswith("w2.TIF")] # only red channel images
imgs = []
for img_file in img_files:
    img_path = os.path.join(img_folder, img_file)
    img = Image.open(img_path)
    img = np.array(img)
    imgs.append(img)
img = np.stack(imgs, axis=-1) # now the last dimension is the channel dimension
print(img.shape)
app.load_image_data(img) # load as numpy array

In [ ]:
# read all red-ch images into arrays and concatenate them in the layer dimension
img_folder = "images/cellular_dataset/single_channel_16bit/"
img_files = sorted(os.listdir(img_folder))
img_files = [f for f in img_files if f.endswith("w2.TIF")] # only red channel images
imgs = []
for img_file in img_files:
    img_path = os.path.join(img_folder, img_file)
    img = Image.open(img_path)
    img = np.array(img)[..., None] # add a new dimension for channels
    imgs.append(img)
img = np.stack(imgs, axis=-1) # now the last dimension is the layer dimension
print(img.shape)
app.load_image_data(img) # load as numpy array

In [ ]:
# combine red and green channels and all layers
img_folder = "images/cellular_dataset/single_channel_16bit/"
img_files = sorted(os.listdir(img_folder))
imgs_red = [f for f in img_files if f.endswith("w2.TIF")] # only red channel images
imgs_green = [f for f in img_files if f.endswith("w1.TIF")] # only green channel images
imgs = []
for img_red, img_green in zip(imgs_red, imgs_green):
    img_path_red = os.path.join(img_folder, img_red)
    img_path_green = os.path.join(img_folder, img_green)
    img_red = Image.open(img_path_red)
    img_green = Image.open(img_path_green)
    img_red = np.array(img_red)
    img_green = np.array(img_green)
    img = np.stack([img_red, img_green], axis=-1) # now the last dimension is the channel dimension
    imgs.append(img)
img = np.stack(imgs, axis=-1) # now the last dimension is the layer dimension
print(img.shape)
app.load_image_data(img) # load as numpy array

In [ ]:
# now use all images in the folder
img_folder = "images/cellular_dataset/single_channel_16bit/"
img_files = sorted(os.listdir(img_folder))
imgs_red = [f for f in img_files if f.endswith("w2.TIF")] # only red channel images
imgs_green = [f for f in img_files if f.endswith("w1.TIF")] # only green channel images
imgs_blue = [f for f in img_files if f.endswith("w3.TIF")] # only blue channel images
imgs = []
for img_red, img_green, img_blue in zip(imgs_red, imgs_green, imgs_blue):
    img_path_red = os.path.join(img_folder, img_red)
    img_path_green = os.path.join(img_folder, img_green)
    img_path_blue = os.path.join(img_folder, img_blue)
    img_red = Image.open(img_path_red)
    img_green = Image.open(img_path_green)
    img_blue = Image.open(img_path_blue)
    img_red = np.array(img_red)
    img_green = np.array(img_green)
    img_blue = np.array(img_blue)
    img = np.stack([img_red, img_green, img_blue], axis=-1) # now the last dimension is the channel dimension
    imgs.append(img)
img = np.stack(imgs, axis=-1) # now the last dimension is the layer dimension
print(img.shape)
app.load_image_data(img) # load as numpy array

## Non-real-time rendering

In [ ]:
app_2 = App()

In [ ]:
# combine red and green channels and all layers
img_folder = "images/cellular_dataset/single_channel_16bit/"
img_files = sorted(os.listdir(img_folder))
imgs_red = [f for f in img_files if f.endswith("w2.TIF")] # only red channel images
imgs_green = [f for f in img_files if f.endswith("w1.TIF")] # only green channel images
imgs = []
for img_red, img_green in zip(imgs_red, imgs_green):
    img_path_red = os.path.join(img_folder, img_red)
    img_path_green = os.path.join(img_folder, img_green)
    img_red = Image.open(img_path_red)
    img_green = Image.open(img_path_green)
    img_red = np.array(img_red)
    img_green = np.array(img_green)
    img = np.stack([img_red, img_green], axis=-1) # now the last dimension is the channel dimension
    imgs.append(img)
img = np.stack(imgs, axis=-1) # now the last dimension is the layer dimension
print(img.shape)
app_2.load_image_data(img) # load as numpy array

In [ ]:
# example minimal setup

# create a feature that will report the mean value of the red channel
mean_red = MeanChannelValue(filter_channels=0, name="MeanRed")
# attach the feature to the app
app_2.attach(mean_red)

# create a Theremin, a simple sine wave synth that we will use to sonify the mean pixel value
theremin = Theremin()
# attach the Theremin to the app
app_2.attach(theremin)

# create a Mapper that will map the mean red pixel value (within the Probe) to the frequency of the Theremin
red2freq = Mapper(
    mean_red, 
    theremin["frequency"], 
    in_low=200,
    in_high=500,
    exponent=2, name="Red2Freq") # cubic mapping curve for a more "linear" feel of frequency changes
# attach the Mapper to the app
app_2.attach(red2freq)

In [ ]:
# example: horizontal scan
duration = 5
my_timeline = [
    (0, {
        "probe_width": 1,
        "probe_height": 500,
        "probe_x": 0,
        "probe_y": 0
    }),
    (duration, {
        "probe_x": 499
    })
]

target_filename = "horizontal_scan.wav"

app_2.render_timeline_to_file(my_timeline, target_filename)

display(Audio(target_filename))

In [ ]:
# example: long horizontal scan, so we reduce app fps to optimize rendering
app_2.fps = 10
duration = 60
my_timeline = [
    (0, {
        "probe_width": 1,
        "probe_height": 500,
        "probe_x": 0,
        "probe_y": 0
    }),
    (duration, {
        "probe_x": 499
    })
]

target_filename = "long_horizontal_scan.wav"

app_2.render_timeline_to_file(my_timeline, target_filename)

display(Audio(target_filename))
app_2.fps = 60

In [ ]:
# example: vertical scan
duration = 5
my_timeline = [
    (0, {
        "probe_width": 500,
        "probe_height": 1,
        "probe_x": 0,
        "probe_y": 0
    }),
    (duration, {
        "probe_y": 499
    })
]

target_filename = "vertical_scan.wav"

app_2.render_timeline_to_file(my_timeline, target_filename)

display(Audio(target_filename))

## Headless mode

In [ ]:
app_3 = App(headless=True)

In [ ]:
app_3.load_image_file("images/test.jpg")

In [ ]:
mean_red_3 = MeanChannelValue(filter_channels=0, name="MeanRed")
app_3.attach(mean_red_3)
theremin_3 = Theremin(name="MySine")
app_3.attach(theremin_3)
red2freq_3 = Mapper(mean_red_3, theremin_3["frequency"], exponent=2, name="Red2Freq")
app_3.attach(red2freq_3)
app_3.features, app_3.synths, app_3.mappers

In [ ]:
app_3.probe_width, app_3.probe_height, app_3.probe_x, app_3.probe_y

In [ ]:
app_3.probe_x, app_3.probe_y = 200, 200

In [ ]:
app_3.audio = True

In [ ]:
app_3.graph.status

In [ ]:
app_3.unmuted = True

In [ ]:
app_3.probe_x, app_3.probe_y = 50, 400

In [ ]:
app_3.unmuted = False

In [ ]:
# a little loop to unmute the probe, then move it around then turn it off
import time
app_3.unmuted = True
app_3.probe_x, app_3.probe_y = 250, 0
while app_3.probe_y < 350:
    app_3.probe_y += 1
    time.sleep(0.01)
app_3.unmuted = False

In [ ]:
app_3.load_image_file("images/cellular_dataset/merged_8bit/Timepoint_001_220518-ST_C03_s1.jpg")

In [ ]:
# a for loop where for each image in the folder we load a headless app and render a timeline in nrt mode
img_folder = "images/cellular_dataset/merged_8bit/"
img_files = sorted(os.listdir(img_folder))

# example: horizontal scan
duration = 5
my_timeline = [
    (0, {
        "probe_width": 1,
        "probe_height": 500,
        "probe_x": 0,
        "probe_y": 0
    }),
    (duration, {
        "probe_x": 499
    })
]
_app = App(headless=True, nrt=True) # create a global graph object, necessary for the Theremin
_app.cleanup() # clean up the app to avoid hanging
# only need to create processor objects once and attach them to the apps
_mean_red = MeanChannelValue(filter_channels=0, name="MeanRed")
_theremin = Theremin(name="MySine")
_red2freq = Mapper(_mean_red, _theremin["frequency"], exponent=2, name="Red2Freq")
# loop over all images in the folder, create a headless app, load the image, attach the processors and render the timeline
for img_file in img_files:
    print(f"Processing {img_file}")
    img_path = os.path.join(img_folder, img_file)
    with App(headless=True, nrt=True) as _app:
        _app.load_image_file(img_path)
        _app.attach(_mean_red)
        _app.attach(_theremin)
        _app.attach(_red2freq)
        target_filename = img_file.replace(".jpg", ".wav")
        _app.render_timeline_to_file(my_timeline, target_filename)
        print(f"Saved {target_filename}")
    display(Audio(target_filename))

In [ ]:
# same thing but render np.arrays instead
img_folder = "images/cellular_dataset/merged_8bit/"
img_files = sorted(os.listdir(img_folder))

# example: horizontal scan
duration = 5
my_timeline = [
    (0, {
        "probe_width": 1,
        "probe_height": 500,
        "probe_x": 0,
        "probe_y": 0
    }),
    (duration, {
        "probe_x": 499
    })
]
_app = App(headless=True, nrt=True) # create a global graph object, necessary for the Theremin
_app.cleanup()
# only need to create processor objects once and attach them to the apps
_mean_red = MeanChannelValue(filter_channels=0, name="MeanRed")
_theremin = Theremin(name="MySine")
_red2freq = Mapper(_mean_red, _theremin["frequency"], exponent=2, name="Red2Freq")
# loop over all images in the folder, create a headless app, load the image, attach the processors and render the timeline
for img_file in img_files:
    print(f"Processing {img_file}")
    img_path = os.path.join(img_folder, img_file)
    buf = None
    with App(headless=True, nrt=True) as _app:
        _app.load_image_file(img_path)
        _app.attach(_mean_red)
        _app.attach(_theremin)
        _app.attach(_red2freq)
        buf = _app.render_timeline_to_array(my_timeline)
        print(f"Saved {target_filename}")
    display(Audio(buf, rate=_app.sample_rate, normalize=False))

This will also work in a script!

## Multichannel Synths

In [ ]:
app_4 = App()

In [ ]:
# combine red and green channels and all layers
img_folder = "images/cellular_dataset/single_channel_16bit/"
img_files = sorted(os.listdir(img_folder))
imgs_red = [f for f in img_files if f.endswith("w2.TIF")] # only red channel images
imgs_green = [f for f in img_files if f.endswith("w1.TIF")] # only green channel images
imgs = []
for img_red, img_green in zip(imgs_red, imgs_green):
    img_path_red = os.path.join(img_folder, img_red)
    img_path_green = os.path.join(img_folder, img_green)
    img_red = Image.open(img_path_red)
    img_green = Image.open(img_path_green)
    img_red = np.array(img_red)
    img_green = np.array(img_green)
    img = np.stack([img_red, img_green], axis=-1) # now the last dimension is the channel dimension
    imgs.append(img)
img = np.stack(imgs, axis=-1) # now the last dimension is the layer dimension
print(img.shape)
app_4.load_image_data(img) # load as numpy array

In [ ]:
# Mean row value (will change the number of features depending of the Probe height)
mean_row = Feature(
    name="Mean Row",
    target_dim=0, # mean value of each row (for horizontal scanning)
)
app_4.attach(mean_row)

In [ ]:
# Mean col value (will change the number of features depending of the Probe width)
mean_col = Feature(
    name="Mean Col",
    target_dim=1, # mean value of each col (for horizontal scanning)
)
app_4.attach(mean_col)

In [ ]:
# create a 16-voice oscillator bank of sawtooth waves, spread across the stereo field
num_voices = 16
freqs = [440 for i in range(num_voices)]
osc_bank = Oscillator(frequency=freqs, panning=[-1, 1], waveform="saw", name="OSCBank") # test multichannel
app_4.attach(osc_bank)

In [ ]:
# create a 16-voice oscillator bank of sawtooth waves, spread across the stereo field
num_voices = 16
freqs = [440 for i in range(num_voices)]
osc_bank_2 = Oscillator(frequency=freqs, panning=[-1, 1], waveform="saw", name="OSCBank2") # test multichannel
app_4.attach(osc_bank_2)

In [ ]:
# Create a mapping where each row value will be mapped to a different channel of the oscillator bank
row2freq = Mapper(mean_row, osc_bank["frequency"], exponent=2, out_high=1000, name="Row2Freq")
app_4.attach(row2freq)

In [ ]:
# Create a mapping where each col value will be mapped to a different channel of the oscillator bank
col2freq = Mapper(mean_row, osc_bank_2["frequency"], exponent=2, out_high=1000, name="Col2Freq")
app_4.attach(col2freq)

## The Feature base class and custom Features

### Feature base class

In [ ]:
app_5 = App()

In [ ]:
# combine red and green channels and all layers
img_folder = "images/cellular_dataset/single_channel_16bit/"
img_files = sorted(os.listdir(img_folder))
imgs_red = [f for f in img_files if f.endswith("w2.TIF")] # only red channel images
imgs_green = [f for f in img_files if f.endswith("w1.TIF")] # only green channel images
imgs = []
for img_red, img_green in zip(imgs_red, imgs_green):
    img_path_red = os.path.join(img_folder, img_red)
    img_path_green = os.path.join(img_folder, img_green)
    img_red = Image.open(img_path_red)
    img_green = Image.open(img_path_green)
    img_red = np.array(img_red)
    img_green = np.array(img_green)
    img = np.stack([img_red, img_green], axis=-1) # now the last dimension is the channel dimension
    imgs.append(img)
img = np.stack(imgs, axis=-1) # now the last dimension is the layer dimension
print(img.shape)
app_5.load_image_data(img) # load as numpy array

In [ ]:
# defaults
base_feature = Feature(
    filter_rows=None,
    filter_columns=None,
    filter_channels=None,
    filter_layers=None,
    target_dim=2,
    reduce_method="mean",
    name="Feature"
)
app_5.attach(base_feature)

In [ ]:
# Mean pixel value
mean_pix = Feature(name="Mean") # defaults to mean pixel value, channel dimension is kept
app_5.attach(mean_pix)

In [ ]:
# Median pixel value
median_pix = Feature(name="Median", reduce_method="median")
app_5.attach(median_pix)

In [ ]:
# Mean row value (will change the number of features depending of the Probe height)
mean_row = Feature(
    name="Mean Row",
    target_dim=0, # mean value of each row (for horizontal scanning)
)
app_5.attach(mean_row)

In [ ]:
# Mean col value (will change the number of features depending of the Probe width)
mean_col = Feature(
    name="Mean Col",
    target_dim=1, # mean value of each col (for horizontal scanning)
)
app_5.attach(mean_col)

In [ ]:
# Median, filter for the red (first) channel only
median_red = Feature(
    name="Median Red",
    target_dim=2, # channel dim
    filter_channels=0, # only use the first channel (red)
    reduce_method="median"
)
app_5.attach(median_red)

### Custom Features

In [ ]:
app_6 = App()

In [ ]:
# combine red and green channels and all layers
img_folder = "images/cellular_dataset/single_channel_16bit/"
img_files = sorted(os.listdir(img_folder))
imgs_red = [f for f in img_files if f.endswith("w2.TIF")] # only red channel images
imgs_green = [f for f in img_files if f.endswith("w1.TIF")] # only green channel images
imgs = []
for img_red, img_green in zip(imgs_red, imgs_green):
    img_path_red = os.path.join(img_folder, img_red)
    img_path_green = os.path.join(img_folder, img_green)
    img_red = Image.open(img_path_red)
    img_green = Image.open(img_path_green)
    img_red = np.array(img_red)
    img_green = np.array(img_green)
    img = np.stack([img_red, img_green], axis=-1) # now the last dimension is the channel dimension
    imgs.append(img)
img = np.stack(imgs, axis=-1) # now the last dimension is the layer dimension
print(img.shape)
app_6.load_image_data(img) # load as numpy array

In [ ]:
# create a custom feature that will return random values
class MyRandomFeature(Feature):
    def __init__(self, name="MyRandomFeature"):
        super().__init__(name=name)

    def process_image(self, mat):
        return np.random.rand(*mat.shape)
    
    def compute(self, mat):
        num_features = mat.shape[self.target_dim]
        return np.random.rand(num_features)
    
my_random_feature = MyRandomFeature()
app_6.attach(my_random_feature)

In [ ]:
app_6.detach(my_random_feature)
my_random_feature = MyRandomFeature()
# change target dim to height (0)
my_random_feature.target_dim = 0
app_6.attach(my_random_feature)

#### K-means example

In [ ]:
# !pip install scikit-learn

In [ ]:
# create a custom feature that will fit a KMeans model to the image and return the histogram of cluster assignments
from sklearn.cluster import KMeans # pip install scikit-learn if you don't have it

class KMeansFeature(Feature):
    def __init__(self, n_clusters=3, name="KMeansFeature"):
        super().__init__(name=name)
        self.n_clusters = n_clusters
        self.kmeans = None
        self._original_shape = None

    def _reshape_for_kmeans(self, mat):
        """Helper to reshape 4D matrix to 2D for KMeans"""
        mat_reshaped = np.moveaxis(mat, self.target_dim, 0)
        return mat_reshaped.reshape(mat_reshaped.shape[0], -1)

    def process_image(self, mat):
        self._original_shape = mat.shape
        features = self._reshape_for_kmeans(mat)
        self.kmeans = KMeans(n_clusters=self.n_clusters).fit(features.T)

        # Get cluster assignments and reshape back to original dimensions
        labels = self.kmeans.predict(features.T)
        other_dims = [s for i, s in enumerate(mat.shape) if i != self.target_dim]
        self.transformed_image = np.expand_dims(
            labels.reshape(*other_dims), 
            axis=self.target_dim
        )
        return self.transformed_image
    
    def compute(self, mat):
        if self.kmeans is None:
            raise ValueError("KMeans model has not been fitted. Call process_image first.")
        features = self._reshape_for_kmeans(mat)
        labels = self.kmeans.predict(features.T)
        # Compute histogram of cluster assignments
        hist, _ = np.histogram(labels, bins=range(self.n_clusters + 1))
        return hist.astype(float) / hist.sum() # normalize to sum to 1

kmeans_feature = KMeansFeature(n_clusters=10)
app_6.attach(kmeans_feature)

In [ ]:
# create a multichannel Theremin that has its frequencies in a harmonic series
fundamental_freq = 110
num_harmonics = kmeans_feature.n_clusters
freqs = fundamental_freq * np.arange(1, num_harmonics + 1)
print("Frequencies:",freqs)
osc = Theremin(frequency=freqs, name="KMeansOsc")
app_6.attach(osc)

# create a Mapper that will map the KMeans cluster histogram to the amplitude of the Theremin 
k2amp = Mapper(kmeans_feature, osc["amplitude"], exponent=1, name="K2Amp")
app_6.attach(k2amp)

In [ ]:
# !pip install matplotlib seaborn

In [ ]:
# visualize the KMeans clusters
import matplotlib.pyplot as plt # pip install matplotlib seaborn if you don't have them
import seaborn as sns

# Create a colormap with distinct colors for each cluster
n_clusters = kmeans_feature.n_clusters
colors = sns.color_palette("husl", n_colors=n_clusters)
colormap = {i: colors[i] for i in range(n_clusters)}

# Get the cluster assignments from transformed_image
cluster_image = kmeans_feature.transformed_image[:, :, 0, 0]
print(cluster_image.shape)

# Create RGB image where each cluster gets a unique color
rgb_image = np.zeros((*cluster_image.shape, 3))
for cluster_id, color in colormap.items():
    mask = cluster_image == cluster_id
    rgb_image[mask] = color

# Plot the results
plt.figure(figsize=(10, 5))

plt.subplot(121)
plt.title('Original Image')
plt.imshow(app_6.image_displayed)  # Show the currently displayed image
plt.axis('off')

plt.subplot(122)
plt.title('K-means Clusters')
plt.imshow(rgb_image)
plt.axis('off')

plt.tight_layout()
plt.show()

## Multiple Apps and the AppRegistry

In [ ]:
from pixasonics.core import AppRegistry
AppRegistry._apps # list of all currently running Apps, should be 6